<font style="font-size:35px;text-align:center;">*CA684 Machine Learning Assignment - Memorability Predictor*</font>

This Notebook has the build of final model used for predicting the test set memorability scores 

I built this predictor based on my exploration done with various features and models. The exploration file is available at code/models_exploration.ipynb

In the below code, I have used captions as main features to predict the scores

Importing the Libraries

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import glob
import os

Importing the Ground Truth Dataset provided for training from CSV file

In [2]:
ground_truth_train = pd.read_csv('dev-set_ground-truth.csv')

In [3]:
ground_truth_train.head()

,video,short-term_memorability,nb_short-term_annotations,long-term_memorability,nb_long-term_annotations
0,video3.webm,0.924,34,0.846,13
1,video4.webm,0.923,33,0.667,12
2,video6.webm,0.863,33,0.700,10
3,video8.webm,0.922,33,0.818,11
4,video10.webm,0.950,34,0.900,10


In [4]:
ground_truth_train = ground_truth_train.drop(['nb_short-term_annotations', 'nb_long-term_annotations'], axis=1)

In [5]:
ground_truth_train.head()

,video,short-term_memorability,long-term_memorability
0,video3.webm,0.924,0.846
1,video4.webm,0.923,0.667
2,video6.webm,0.863,0.700
3,video8.webm,0.922,0.818
4,video10.webm,0.950,0.900


Importing the Ground Truth for test data from CSV file

In [6]:
ground_truth_test = pd.read_csv('testset/ground_truth_template.csv')

In [7]:
ground_truth_test.head()

,video,short-term_memorability,nb_short-term_annotations,long-term_memorability,nb_long-term_annotations
0,7494,NaN,33,NaN,12
1,7495,NaN,34,NaN,10
2,7496,NaN,32,NaN,13
3,7497,NaN,33,NaN,10
4,7498,NaN,33,NaN,10


<font style="font-size:25px;text-align:center;">***Extracting features using Captions***</font>

Importing the training set video caption files

In [8]:
video_captions_train = pd.read_csv('dev-set_video-captions.txt',delimiter='\t',header= None,names=('video','Captions'))

In [9]:
video_captions_train.head()

,video,Captions
0,video3.webm,blonde-woman-is-massaged-tilt-down
1,video4.webm,roulette-table-spinning-with-ball-in-closeup-shot
2,video6.webm,khr-gangsters
3,video8.webm,medical-helicopter-hovers-at-airport
4,video10.webm,couple-relaxing-on-picnic-crane-shot


Importing the test set video caption files

In [10]:
video_captions_test = pd.read_csv('testset/test-set-1_video-captions.txt',delimiter='\t',header= None,names=('video','Captions'))

In [11]:
video_captions_test.head()

,video,Captions
0,video7494.webm,green-jeep-struggling-to-drive-over-huge-rocks
1,video7495.webm,hiking-woman-tourist-is-walking-forward-in-mou...
2,video7496.webm,close-up-of-african-american-doctors-hands-usi...
3,video7497.webm,slow-motion-of-a-man-using-treadmill-in-the-gy...
4,video7498.webm,slow-motion-of-photographer-in-national-park


We need to extract the features using both the train and test sets so that we can train our models with similar dimensions.

In [12]:
video_captions = pd.concat([video_captions_train, video_captions_test],ignore_index=True)

In [13]:
video_captions.head()

,video,Captions
0,video3.webm,blonde-woman-is-massaged-tilt-down
1,video4.webm,roulette-table-spinning-with-ball-in-closeup-shot
2,video6.webm,khr-gangsters
3,video8.webm,medical-helicopter-hovers-at-airport
4,video10.webm,couple-relaxing-on-picnic-crane-shot


In [14]:
video_captions.tail()

,video,Captions
7995,video10004.webm,astronaut-in-outer-space-against-the-backdrop-...
7996,video10005.webm,young-women-lying-on-sunbed-and-applying-sun-c...
7997,video10006.webm,doctor-talking-to-patient-using-a-tablet-to-ex...
7998,video10007.webm,businessman-sitting-on-the-beach-on-inflatable...
7999,video10008.webm,woman-eating-ice-cream-and-sitting-in-the-stre...


Cleaning the text data of Captions using NLP Libraries

Importing the NLP Libraries

In [15]:
import re
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/swathikiran/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


Giving weights to certain words based on their positive and negetive coefficients in R Gupta et al. papaer (2018)

Giving lower values to negetive coefficient words ( weights chose in range between 0 and 0.20)

In [16]:
negavtive_weights = {'mountains':0.01,'sunset':0.02,'trees':0.03,'mountain':0.04,'clouds':0.05,'river':0.06,'aerialof':0.07,'lake':0.08,'forest':0.09,'kenya':0.10,'landscape':0.11,'valley':0.12,'leaves':0.13,'sun':0.14}

Giving higher values to positive coefficient words ( weights chose in range between 0.50 and 0.70)

In [17]:
positive_weights = {'woman':0.65,'eating':0.64,'putting':0.63,'lying':0.62,'girl':0.61,'selfie':0.60,'relaxing':0.59,'jellyfish':0.58,'cat':0.57,'super':0.56,'slow':0.55,'super':0.54,'american':0.53,'potrait':0.52,'pregnant':0.51,'couple':0.50}

Removing the special charecters and retaining only text data in lowercase text,
Removing Stopwords,
Storing the cleaned text data into a list named captions

Assigning weights accordingly if certain words exists in the captions

In [18]:
cleaned_captions = []
pos_list = []
neg_list = []
for i in range(0, 8000):
    local_pos_weight = 0
    local_neg_weight = 0
    text = re.sub('[^a-zA-Z]', ' ', video_captions['Captions'][i])
    text = text.lower()
    text = text.split()
    text = [word for word in text if not word in set(stopwords.words('english'))]
    for word in text:
        if(word in set(positive_weights.keys())):
                pw = local_pos_weight + positive_weights[word]
        if(word in set(negavtive_weights.keys())):
                nw = local_neg_weight + negavtive_weights[word]
    pos_list.append(local_pos_weight)
    neg_list.append(local_neg_weight)
    text = ' '.join(text)
    cleaned_captions.append(text)

In [19]:
weights_df = pd.DataFrame(
    {'positive_weights': pos_list,
     'negative_weights': neg_list
    })

In [20]:
weights_df.shape

(8000, 2)

In [21]:
len(cleaned_captions)

8000

Extracting the TFIDF score features from the cleaned captions

In [22]:
from sklearn.feature_extraction.text import TfidfVectorizer
tf = TfidfVectorizer()

In [23]:
caption_features = tf.fit_transform(cleaned_captions).toarray()

In [24]:
caption_features.shape

(8000, 5762)

In [25]:
CaptionAndWeight_features = np.concatenate((caption_features, weights_df), axis=1)

In [26]:
caption_features_train = CaptionAndWeight_features[:6000]

In [27]:
caption_features_test = CaptionAndWeight_features[6000:]

In [28]:
caption_features_train.shape

(6000, 5764)

In [29]:
caption_features_test.shape

(2000, 5764)

<font style="font-size:25px;text-align:center;">**Defining Get_Score method to calculate the Spearmen's Score**</font>

In [30]:
def Get_score(Y_pred,Y_true):
    '''Calculate the Spearmann"s correlation coefficient'''
    Y_pred = np.squeeze(Y_pred)
    Y_true = np.squeeze(Y_true)
    if Y_pred.shape != Y_true.shape:
        print('Input shapes don\'t match!')
    else:
        if len(Y_pred.shape) == 1:
            Res = pd.DataFrame({'Y_true':Y_true,'Y_pred':Y_pred})
            score_mat = Res[['Y_true','Y_pred']].corr(method='spearman',min_periods=1)
            print('The Spearman\'s correlation coefficient is: %.3f' % score_mat.iloc[1][0])
        else:
            for ii in range(Y_pred.shape[1]):
                Get_score(Y_pred[:,ii],Y_true[:,ii])

<font style="font-size:25px;text-align:center;">***Building the Model***</font>

Passing these parameters for predicting shortterm memorability only

In [38]:
X= caption_features_train
y = ground_truth_train.iloc[:, 1:3].values

Splitting the dataset into the Training set and Validation set

In [39]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.20)

Fitting Random Forest Regression model on Captions with TFIDF

In [40]:
from sklearn.ensemble import RandomForestRegressor
RFregressor = RandomForestRegressor()
RFregressor.fit(X_train, y_train)

RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
           oob_score=False, random_state=None, verbose=0, warm_start=False)

In [42]:
y_pred = RFregressor.predict(X_test)
Get_score(y_pred, y_test)

The Spearman's correlation coefficient is: 0.340
The Spearman's correlation coefficient is: 0.137


Retraining the model with full training data with best parameters

In [43]:
X= caption_features_train
y = ground_truth_train.iloc[:, 1:3].values

In [44]:
X_test = caption_features_test

In [ ]:
from sklearn.ensemble import RandomForestRegressor
RFregressor = RandomForestRegressor(n_estimators = 150)
RFregressor.fit(X, y)

In [ ]:
y_pred = RFregressor.predict(X_test)

In [ ]:
predictions = np.asarray(y_pred)
np.savetext("predictions.csv", predictions, delimiter = ",")